In [3]:
import psycopg2
import pandas as pd
import json
import os
from dotenv import load_dotenv

# Convert CSV into DataFrame format

In [4]:
#Convert CSV File W/ Data into dataframe, make sure you have the correct file and in the proper csv format
df = pd.read_csv("https://raw.githubusercontent.com/JonRivera/Bridges_Prosperity_Labs28_Project/main/Data/cleaned_merged_dataset_final_revised.csv")

In [5]:
#Data frame, had a duplicated index column so I removed it
df = df.iloc[:, 1:]

In [6]:
#Data frame looks good to go
df.head()

,bridge_name,bridge_opportunity_project_code,bridge_opportunity_needs_assessment,bridge_opportunity_level1_government,bridge_opportunity_level2_government,bridge_opportunity_stage,bridge_opportunity_gps_latitude,bridge_opportunity_gps_longitude,bridge_opportunity_bridge_type,bridge_opportunity_span_m,...,bridge_classification,flag_for_rejection,rejection_reason,bridge_type,estimated_span_m,height_differential_between_banks,bridge_opportunity_general_project_photos,bridge_opportunity_casesafeid,senior_engineering_review_conducted,country
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,Cancelled,-2.760833,29.488056,Suspended Bridge,NaN,...,NaN,Yes,There is a planned vehicular road with a vehic...,NaN,NaN,NaN,NaN,006f100000asFErAAM,Yes,Rwanda
1,Kagarama,1007325,NaN,Western Province,Nyamasheke,Complete,-2.322534,29.141944,Suspension Bridge,48.0,...,NaN,NaN,NaN,NaN,10.0,0.7,https://flic.kr/s/aHskvedfPB,006f100000a82QzAAI,Yes,Rwanda
2,Karambi,1007374,NaN,Southern Province,Huye,Complete,-2.513333,29.612778,Suspension Bridge,50.0,...,Standard,NaN,NaN,Suspension,45.0,1.0,https://flic.kr/s/aHskbMF1Kr,006f100000a86DaAAI,Yes,Rwanda
3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.476407,29.420800,Suspension Bridge,68.0,...,Standard,No,NaN,Suspended,47.4,1.1,https://www.flickr.com/photos/bridgestoprosper...,006f100000a86F4AAI,Yes,Rwanda
4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.233022,29.575130,Suspension Bridge,78.0,...,Standard,No,NaN,Suspended,90.0,1.1,https://www.flickr.com/photos/bridgestoprosper...,006f100000a86FXAAY,Yes,Rwanda


# Clean Columns If Necessary

In [7]:
#Get column names, having clean column names are important to have to make consisten and repoducible queries
#Specifically make sure you don't have white spaces, numbers at the start, or () brackets within the column name
#May encounter errors when running queries on postgres sql database, or also may run into errors into convertin data frame
#Into SQL Table
columns = list(df.columns)

In [12]:
columns

['bridge_name',
 'bridge_opportunity_project_code',
 'bridge_opportunity_needs_assessment',
 'bridge_opportunity_level1_government',
 'bridge_opportunity_level2_government',
 'bridge_opportunity_stage',
 'bridge_opportunity_gps_latitude',
 'bridge_opportunity_gps_longitude',
 'bridge_opportunity_bridge_type',
 'bridge_opportunity_span_m',
 'bridge_opportunity_individuals_directly_served',
 'bridge_opportunity_comments',
 'form_form_name',
 'form_created_by',
 'proposed_bridge_location_gps_latitude',
 'proposed_bridge_location_gps_longitude',
 'current_crossing_method',
 'nearest_all_weather_crossing_point',
 'days_per_year_river_is_flooded',
 'flood_duration_during_rainy_season',
 'market_access_blocked_by_river',
 'education_access_blocked_by_river',
 'health_access_blocked_by_river',
 'other_access_blocked_by_river',
 'primary_occupations',
 'primary_crops_grown',
 'river_crossing_deaths_in_last_3_years',
 'river_crossing_injuries_in_last_3_years',
 'incident_descriptions',
 'notes_o

# Check Secrets

In [8]:
#Secrets are contained in a .envfile
load_dotenv()
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# print(db_name)
# print(db_user)
# print(db_password)
# print(db_host)
# print(db_port)

# Verify which tables exist in DataBase

In [10]:
import sqlalchemy  # Package for accessing SQL databases via Python
# Connect to database (Note: The package psychopg2 is required for Postgres to work with SQLAlchemy)
engine = sqlalchemy.create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/bridges_to_prosperity_oct2018")
con = engine.connect()
# Verify that there are no existing tables
print(engine.table_names())
engine.dispose()

['B2P_Oct_2018', 'B2P_OCT_2018_Merged', 'B2P_oct_2018']


# Upload DataFrame as SWL Table to DataBase

In [11]:
# table_name = 'B2P_Merged_Final'
# df.to_sql(table_name, con)

In [13]:
def conn_curs():
    """
    makes a connection to the database
    """
    global db_name
    global db_user
    global db_password
    global db_host
    global db_port
    
    connection = psycopg2.connect(dbname=db_name, user= db_user,
                                  password=db_password, host= db_host,port=db_port)
    cursor = connection.cursor()
    return connection, cursor

In [16]:
def fetch_query_records(query):
    global conn_curs
    conn, cursor = conn_curs()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    return result

In [17]:
fetch_query_records("""SELECT "bridge_name" from public."B2P_Merged_Final" LIMIT 1;""")

[('Bukinga',)]

In [20]:
# Testing Querie to get Records based on Bridge Name
conn, cursor = conn_curs()
query  = """SELECT "bridge_name" from public."B2P_Merged_Final" where "bridge_name" = 'Bukinga' LIMIT 1;"""
cursor.execute(query)
result = cursor.fetchall()
cursor.close()
conn.close
print(result)

[('Bukinga',)]


In [27]:
def fetch_all_records():
    global conn_curs
    conn,cursor = conn_curs()
    query = """ SELECT * from public."B2P_Merged_Final" LIMIT 1""";
    cursor.execute(query)
    result = cursor.fetchall() #when it fetches it adds an extra column (indexes), make sure to remove it
    conn.close()
    df2 = pd.DataFrame(result, columns= ['index']+columns)
    df2 = df2.iloc[:,1:] # This code removes extra copied indexed column
    df_json = df.to_json(orient='records')
    parsed = json.loads(df_json)
    return df2


In [28]:
#Testing fetch_all_records function
fetch_all_records()

,bridge_name,bridge_opportunity_project_code,bridge_opportunity_needs_assessment,bridge_opportunity_level1_government,bridge_opportunity_level2_government,bridge_opportunity_stage,bridge_opportunity_gps_latitude,bridge_opportunity_gps_longitude,bridge_opportunity_bridge_type,bridge_opportunity_span_m,...,bridge_classification,flag_for_rejection,rejection_reason,bridge_type,estimated_span_m,height_differential_between_banks,bridge_opportunity_general_project_photos,bridge_opportunity_casesafeid,senior_engineering_review_conducted,country
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,Cancelled,-2.760833,29.488056,Suspended Bridge,None,...,None,Yes,There is a planned vehicular road with a vehic...,None,None,None,None,006f100000asFErAAM,Yes,Rwanda
